In [61]:
'''
程式邏輯
(三個def)

a. def create_ppt_from_images (路徑, 簡報檔名稱)
   建立prs物件，設定長寬。
   辨識非為文字雲的圖表，根據每一張符合條件的.png，並執行add_image_slide。
b. def add_image_slide(投影片物件，圖片路徑)
   符合條件的.png，先建立新的投影片
   設定圖片大小與位置，添加圖片
   設定文字框位置與字體，添加投影片標題(標題來自generate_title)
c. generate_title(檔案名稱)(已包含在add_image_slide)
   到這裡來的檔案已經在add_image_slide處理過了，會只剩下檔名(os.path.basename())
   剔除擴展名並透過"_"切割
   擷取第一([0])和第三([2])檔名作為投影片標題

直接執行create_ppt_from_images,貼上對應路徑與簡報檔名稱

待優化：
1. 自動化後，需手動開啟投影片母片(因為add_image_slide直接使用佈景主題，圖片位置會不對，需進一步調整參數)

解決方法：
目前會直接得到空白佈景的完成檔，直接將其複製到有主題的ppt中，速度很快，花時間解決投影片母片問題反倒效率較低。

'''


import os
from pptx import Presentation
from pptx.util import Cm, Pt
from pptx.dml.color import RGBColor

# for one image/slide
def generate_title(filename): # filename 來自後面create_ppt_from_images的迴圈內符合條件的
    name_without_ext = os.path.splitext(filename)[0] # 也可使用 method chaining，os.path.splitext會區分文件名與擴展名(.png)，[0]就是只有擷取文件名
    parts = name_without_ext.split('_') # split the path by "_"
    if len(parts) >= 3: # the file is the target if parts>3 
        return f"{parts[0]}_{parts[2]}主題" # 第一個return後就不會執行下面程式
    return name_without_ext # if parts <= 3 then return original


# for one image/slide
def add_image_slide(prs, image_path): # main target: create a blank slide then add png, title
    blank_slide_layout = prs.slide_layouts[6]  # 使用空白主題(預設第七頁是空白的)
    slide = prs.slides.add_slide(blank_slide_layout) # 加入一張簡報(slides.add.slide)
    
    # 設置圖片大小和位置
    width = height = Cm(18)  # 18公分
    left = Cm(8.23)  # 水平位置8.23公分，從左上角
    top = Cm(2)      # 垂直位置2公分，從左上角
    
    # 添加圖片並設置其大小和位置
    pic = slide.shapes.add_picture(image_path, left, top, width, height)  # slide.shape.add_picture 為加入指定圖片(路徑, 大小與位置)
    
    # 添加標題文本框（更新位置和大小）
    title_left = Cm(1.72)    # 水平位置1.72公分，從左上角
    title_top = Cm(1.05)     # 垂直位置1.05公分，從左上角
    title_width = Cm(31.01)  # 寬度31.01公分
    title_height = Cm(1.92)  # 高度1.92公分
    
    title_shape = slide.shapes.add_textbox(title_left, title_top, title_width, title_height)  # add_textbox為加入空白方塊(大小與繪製)
    title_frame = title_shape.text_frame # 設置文字框屬性
    title_frame.word_wrap = True # 框內自動換行
    
    p = title_frame.paragraphs[0] #文字框內的第一個段落([0])，基本上一開始設置也會只有第一個段落
    p.text = generate_title(os.path.basename(image_path)) # 產生標題，返回路徑str中最後一組(也就是檔案名稱含擴展名)
    p.alignment = 1  # 置中對齊
    
    # 設置標題格式
    font = p.font #以下都是設置字體的屬性
    font.name = '微軟正黑體'
    font.size = Pt(40)
    font.color.rgb = RGBColor(0, 0, 0)  # 黑色

def create_ppt_from_images(image_folder, output_ppt): # this def is about create a ppt and exclude the png which is not wordcloud 
    
    prs = Presentation() # create new ppt object
     # 設置幻燈片大小為標準16:9
    prs.slide_width = Cm(33.867)  # 16:9 寬度
    prs.slide_height = Cm(19.05)  # 16:9 高度
    
    for filename in os.listdir(image_folder): #　os.listdir() will return a list of all the file in the folder(only filename)
        if filename.lower().endswith('.png') and '主題差異' not in filename: # setting the condition of ta png, 1.end = "png." after convert to lowerkey 2.'主題差異' not in filename
            image_path = os.path.join(image_folder, filename) # if the file is ta, join path and name (using function os.path.join)
            add_image_slide(prs, image_path) # connect to the next def - add_image_slide(every file iterate, hence it will pass every add_image_slide and generate_title)
    prs.save(output_ppt)

# TA file
image_folder = "folder_path"
output_ppt = "ppt_path"

create_ppt_from_images(image_folder, output_ppt)

print(f"PowerPoint 已生成：{output_ppt}")

PowerPoint 已生成：C:\Users\40050\Desktop\\output_presentation.pptx
